# Part 2 - Shortest Path Algorithms

## Introduction - 
Djikstra's Shortest Path Algorithm and A* Algorithm

### Pseudocode

In [1]:
from Part2_Graph import Graph

In [21]:
graph = Graph()
graph.add_node('A')
graph.add_node('B')
graph.add_node('C')
graph.add_node('D')
graph.add_node('E')

graph.add_edge('A','B', 1)
graph.add_edge('A','C', 1)
graph.add_edge('A','E', 1)
graph.add_edge('B','C', 1)
graph.add_edge('B','E', 1)

graph.is_empty
graph.num_nodes
graph.num_edges
graph.nodes
graph.edges
graph.neighbors('A')
graph.neighbors('D')
graph.adjacent('A','B')
graph.adjacent('A','D')

False
5
5
['A', 'B', 'C', 'D', 'E']
[A - B: 1, A - C: 1, A - E: 1, B - C: 1, B - E: 1]
['B', 'C', 'E']
[]
True
False


In [23]:
e_graph = Graph()

print(e_graph.is_empty)
print(e_graph.num_nodes)
print(e_graph.num_edges)
print(e_graph.nodes)
print(e_graph.edges)
# print(e_graph.neighbors('A'))
# print(e_graph.neighbors('D'))
# print(e_graph.adjacent('A','B'))
# print(e_graph.adjacent('A','D'))

True
0
0
[]
[]
